# JAXLOB Introduction Notebook

#### This file will demonstrate, and visualise the key functionalities of the JAXLOB order book implementation. Further, it will measure the walltime speeds for these basic operations.

In [3]:
%load_ext autoreload
%autoreload 2
from functools import partial, partialmethod
from typing import OrderedDict
from jax import numpy as jnp
import jax
import numpy as np
import gymnax_exchange.jaxob.JaxOrderBookArrays as job
import random
import time
import timeit

import sys
# ******** INSERT PATH HERE ********
sys.path.append('/Users/sasrey/AlphaTrade')
import gymnax_exchange




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Helper functions to create (fictional) initial orderbook states and to easily create messages in the LOBSTER format for the book to process. 

In [5]:
def create_init_book(order_capacity=10,trade_capacity=10,pricerange=[2190000,2200000,2210000],quantrange=[0,500],timeinit=[34200,0]):
    """
    Generates a random orderbook state for a given maximum capacity for orders and trades. 
    Random prices/quantities generated by uniform sampling in pricerange/quantrange. 
    """
    qtofill=order_capacity//3 #fill one third of the available space
    asks=[]
    bids=[]
    orderid=1000
    traderid=1000
    times=timeinit[0]
    timens=timeinit[1]
    for i in range(qtofill):
        asks.append([random.randint(pricerange[1],pricerange[2]),random.randint(quantrange[0],quantrange[1]),orderid,traderid,times,timens])
        times+=random.randint(0,1)
        timens+=random.randint(0,10000)
        bids.append([random.randint(pricerange[1],pricerange[2]),random.randint(quantrange[0],quantrange[1]),orderid+1,traderid+1,times,timens])
        times+=random.randint(0,1)
        timens+=random.randint(0,10000)
        orderid+=2
        traderid+=2
    bids=jnp.concatenate((jnp.array(bids),jnp.ones((order_capacity-qtofill,6),dtype=jnp.int32)*-1),axis=0)
    asks=jnp.concatenate((jnp.array(asks),jnp.ones((order_capacity-qtofill,6),dtype=jnp.int32)*-1),axis=0)
    trades=jnp.ones((trade_capacity,6),dtype=jnp.int32)*-1
    return asks,bids,trades

def create_message(type='limit',side='bid',price=2200000,quant=10,times=36000,timens=0):
    """
    Generates a specific message (based on human-editable inputs)
    Outputs both the 'dictionary' format 
    """
    if type=='limit':
        type_num=1
    elif type =='cancel' or type == 'delete':
        type_num=2
    elif type =='market':
        type_num=4
    else:
        raise ValueError('Type is none of: limit, cancel, delete or market')

    if side=='bid':
        side_num=1
    elif side =='ask':
        side_num=-1
    else:
        raise ValueError('Side is none of: bid or ask')
    
    dict_msg={
        'side':side_num,
        'type':type_num,
        'price':price,
        'quantity':quant,
        'orderid':8888,
        'traderid':8888,
        'time':times,
        'time_ns':timens}
    array_msg=jnp.array([type_num,side_num,quant,price,8888,8888,times,timens])
    return dict_msg,array_msg

def create_message_forvmap(type='limit',side='bid',price=2200000,quant=10,times=36000,timens=0,nvmap=10):
    if type=='limit':
        type_num=1
    elif type =='cancel' or type == 'delete':
        type_num=2
    elif type =='market':
        type_num=4
    else:
        raise ValueError('Type is none of: limit, cancel, delete or market')

    if side=='bid':
        side_num=1
    elif side =='ask':
        side_num=-1
    else:
        raise ValueError('Side is none of: bid or ask')
    
    dict_msg={
    'side':jnp.array([side_num]*nvmap),
    'type':jnp.array([type_num]*nvmap),
    'price':jnp.array([price]*nvmap),
    'quantity':jnp.array([quant]*nvmap),
    'orderid':jnp.array([8888]*nvmap),
    'traderid':jnp.array([8888]*nvmap),
    'time':jnp.array([times]*nvmap),
    'time_ns':jnp.array([timens]*nvmap)}
    array_msg=jnp.array([type_num,side_num,quant,price,8888,8888,times,timens]*nvmap)
    return dict_msg,array_msg

Global variables used for timing operations. Under 'timeit' function, the target code is run and timed for n_runs, then this operation is repeated for n_repeats. 

The vector of capacities is used to illustrate the effect of the maximum number of orders and trades which can be stored in their respective arrays.

nvmap illustrates the number of book that are processed in parallel when using the vmap operator. 

In [6]:
n_runs=1000
n_repeats=10
order_and_trade_capacities=[10,100,1000]
vmap_tests=False
nvmap=1000

## ADD operation

The add operation (add_order) is called when a limit order is submitted and has some remaining unmatched quantitiy after matching. 

In [19]:
## Add an order

random.seed(0)
addout=[]
bid_inits=[]
for s in order_and_trade_capacities:
    asks,bids,trades=create_init_book(order_capacity=s)
    bid_inits.append(bids)
    mdict,marray=create_message(type='limit',side='bid',price=2191200,quant=77)
    out=job.add_order(bids,mdict)
    addout.append(out)
    res=np.array(timeit.repeat('val=job.add_order(bids,mdict); jax.block_until_ready(val)',repeat=n_repeats,number=n_runs,globals=globals()))
    mu=np.mean(res/n_runs)
    sigma=np.std(res/n_runs)
    mi=np.min(res/n_runs)
    print("Add time for orderbook of size",s,":",mu,"Stdev: ", sigma," Min: ",mi)

print("\n Bid side after initialisation")
print(bid_inits[0])
print("\n Bid side state after adding an order.")
print(addout[0])





if vmap_tests:
    random.seed(0)
    for i,s in enumerate(order_and_trade_capacities):
        asks,bids,trades=create_init_book(order_capacity=s)
        vmdict,marray=create_message_forvmap(type='limit',side='bid',price=2191200,quant=77,nvmap=nvmap)
        vbids=jnp.array([bids]*nvmap)
        out=job.add_order(bids,mdict)
        outv=jax.vmap(job.add_order,(0,{'orderid': 0, 'price': 0, 'quantity': 0, 'side': 0, 'time': 0, 'time_ns': 0, 'traderid': 0, 'type': 0}))(vbids,vmdict)
        res=np.array(timeit.repeat("val=jax.vmap(job.add_order,(0,{'orderid': 0, 'price': 0, 'quantity': 0, 'side': 0, 'time': 0, 'time_ns': 0, 'traderid': 0, 'type': 0}))(vbids,vmdict); jax.block_until_ready(val)",number=n_runs,repeat=n_repeats,globals=globals()))
        mu=np.mean(res/n_runs)
        sigma=np.std(res/n_runs)
        mi=np.min(res/n_runs)
        print("VMAP add time for orderbook of size",s," \n various incoming order sizes:",mu,"Stdev: ", sigma," Min: ",mi)






Add time for orderbook of size 10 : 0.00011737249754369261 Stdev:  3.0824021586964605e-06  Min:  0.00011332417652010917
Add time for orderbook of size 100 : 0.00013630799278616904 Stdev:  2.4911982493466934e-06  Min:  0.00013347069919109343
Add time for orderbook of size 1000 : 0.00017689681947231293 Stdev:  1.764358426550543e-06  Min:  0.0001742841489613056

 Bid side after initialisation
[[2204242     494    1001    1001   34201     663]
 [2209558     456    1003    1003   34203   13163]
 [2204104     465    1005    1005   34203   22984]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]]

 Bid side state after adding an order.
[[2204242     494    1001    1001   34201     663]
 [220

# CANCEL operation

Removes the desired quantity at the given price level. If the quantity is totally removed, removes the order and frees up the space. 

In [21]:
random.seed(0)
cancelout=[]
for i,s in enumerate(order_and_trade_capacities):
    bids=addout[i]
    mdict,marray=create_message(type='cancel',side='bid',price=2191200,quant=77)
    out=job.cancel_order(bids,mdict)
    cancelout.append(out)
    res=np.array(timeit.repeat('val=job.cancel_order(bids,mdict); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
    mu=np.mean(res/n_runs)
    sigma=np.std(res/n_runs)
    print("Cancel time for orderbook of size",s,":",mu,"Stdev: ", sigma)

print("\n Bid side state after cancellation")
print(cancelout[0])

Cancel time for orderbook of size 10 : 9.196944460272788e-05 Stdev:  3.3609434477458553e-06
Cancel time for orderbook of size 100 : 0.00012628505080938338 Stdev:  1.0348925923892532e-06
Cancel time for orderbook of size 1000 : 0.00013958838433027266 Stdev:  1.0372547727342374e-06

 Bid side state after cancellation
[[2204242     494    1001    1001   34201     663]
 [2209558     456    1003    1003   34203   13163]
 [2204104     465    1005    1005   34203   22984]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]]


# MATCH operation
Matching a single very large order against an identified order (and price) from the other side of the book. There will still be a remaining quantity to match after this matching operation.

In [22]:
matchout=[]
for i,s in enumerate(order_and_trade_capacities):
    #Doesn't actually get used: just initialisation of empty list of trades. 
    _,_,trades=create_init_book(order_capacity=s)

    bids=cancelout[i]
    idx=job._get_top_bid_order_idx(bids)
    print("Time to get top bid order for order book of size ",s,":",timeit.timeit('val=job._get_top_bid_order_idx(bids); jax.block_until_ready(val)',number=n_runs,globals=globals())/n_runs)

    matchtuple=(idx,bids,1000,0,trades,9999,36000,1)
    bids,qtm,price,trades,agrid,times,timens=job.match_order(matchtuple)
    
    matchout.append((bids,qtm,trades))
    res=np.array(timeit.repeat('val=job.match_order(matchtuple); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
    mu=np.mean(res/n_runs)
    sigma=np.std(res/n_runs)
    print("Match time for orderbook of size",s,":",mu,"Stdev: ", sigma)


print("\n Bid side state after matching the order.")
print(matchout[0][0])
print("\n Trades logged after the matching operation")
print(matchout[0][2])
print("\n Quantity still to match (or add) from the incoming order after matching this order")
print(matchout[0][1])

Time to get top bid order for order book of size  10 : 4.305707663297653e-05
Match time for orderbook of size 10 : 0.00017666900455951692 Stdev:  4.7000470129610035e-06
Time to get top bid order for order book of size  100 : 8.734332770109176e-05
Match time for orderbook of size 100 : 0.00018013015836477282 Stdev:  1.0935386424906425e-05
Time to get top bid order for order book of size  1000 : 9.613094106316567e-05
Match time for orderbook of size 1000 : 0.00016761468797922137 Stdev:  3.369443285157917e-07

 Bid side state after matching the order.
[[2204242     494    1001    1001   34201     663]
 [     -1      -1      -1      -1      -1      -1]
 [2204104     465    1005    1005   34203   22984]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1      -1      -1]
 [     -1      -1      -1      -1    

Match against an entire side, by using a price of 0 (inf if matching against ask orders). 

In this case the incoming quantity will have an effect on the time to match. 

In [16]:
matchout=[]

for i,s in enumerate(order_and_trade_capacities):
    for j,q in enumerate([0,10,500,1000,10000]):
        _,_,trades=create_init_book(order_capacity=s,trade_capacity=s)

        bids=cancelout[i]

        matchtuple=(bids,q,0,trades,9999,36000,1)
        bids,qtm,price,trades=job._match_against_bid_orders(*matchtuple)
        
        matchout.append((bids,qtm,trades))
        res=np.array(timeit.repeat('val=job._match_against_bid_orders(*matchtuple); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
        mu=np.mean(res/n_runs)
        sigma=np.std(res/n_runs)
        print("Match time for orderbook of size",s," \n with an incoming order of size",q,":",mu,"Stdev: ", sigma)

if vmap_tests:
    for i,s in enumerate(order_and_trade_capacities):
        _,_,trades=create_init_book(order_capacity=s,trade_capacity=s)
        bids=cancelout[i]

        vbids=jnp.array([bids]*nvmap)
        vtrades=jnp.array([trades]*nvmap)
        vq=jnp.array([100,100,100,100,100]*(nvmap//5))

        matchtuple=(vbids,vq,0,vtrades,9999,36000,1)
        jax.vmap(job._match_against_bid_orders,((0,0,None,0,None,None,None)))(*matchtuple)
        
        matchout.append((bids,qtm,trades))
        res=np.array(timeit.repeat('val=jax.vmap(job._match_against_bid_orders,((0,0,None,0,None,None,None)))(*matchtuple); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
        mu=np.mean(res/n_runs)
        sigma=np.std(res/n_runs)
        
        print("VMAP Match time for orderbook of size",s," \n various incoming order sizes:",mu,"Stdev: ", sigma)




Match time for orderbook of size 10  
 with an incoming order of size 0 : 0.0001599755199626088 Stdev:  1.4136404594710928e-05
Match time for orderbook of size 10  
 with an incoming order of size 10 : 0.00023149909507483242 Stdev:  8.236890279002566e-07
Match time for orderbook of size 10  
 with an incoming order of size 500 : 0.00023494059592485433 Stdev:  3.7953098015473232e-06
Match time for orderbook of size 10  
 with an incoming order of size 1000 : 0.00024334206711500885 Stdev:  1.356343510487945e-05
Match time for orderbook of size 10  
 with an incoming order of size 10000 : 0.00025096572581678625 Stdev:  1.0152672097264163e-05
Match time for orderbook of size 100  
 with an incoming order of size 0 : 0.0002072071246802807 Stdev:  2.8212552781626246e-06
Match time for orderbook of size 100  
 with an incoming order of size 10 : 0.0003378016976639629 Stdev:  1.546980475443614e-06
Match time for orderbook of size 100  
 with an incoming order of size 500 : 0.000333969629369676

# High - Level Message Types

## Passive Limit Orders
## Cancel Orders
## Aggressive Limit Orders



In [30]:
random.seed(0)
outs=[]
for i in order_and_trade_capacities:
    asks,bids,trades=create_init_book(order_capacity=i,trade_capacity=i)
    _,limitmsg=create_message(type='limit',side='bid',price=2191200,quant=77)
    _,cancelmsg=create_message(type='cancel',side='bid',price=2191200,quant=77)
    _,matchmsg=create_message(type='limit',side='ask',price=2191200,quant=100)

    bid_limit,_=job.cond_type_side((asks,bids,trades),limitmsg)
    res=np.array(timeit.repeat('val=job.cond_type_side((asks,bids,trades),limitmsg); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
    mu=np.mean(res/n_runs)
    sigma=np.std(res/n_runs)
    print("Limit order for book of size ",i,":",mu,"Stdev: ", sigma)
    bid_cancel,_=job.cond_type_side(bid_limit,cancelmsg)
    res=np.array(timeit.repeat('val=job.cond_type_side(out,cancelmsg); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
    mu=np.mean(res/n_runs)
    sigma=np.std(res/n_runs)
    print("Cancel order for book of size ",i,":",mu,"Stdev: ", sigma)
    bid_cross_limit,_=job.cond_type_side(bid_cancel,matchmsg)
    res=np.array(timeit.repeat('val=job.cond_type_side(out,matchmsg); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
    mu=np.mean(res/n_runs)
    sigma=np.std(res/n_runs)
    print("Matched limit order for book of size ",i,":",mu,"Stdev: ", sigma)
    outs.append((bids,bid_limit[1],bid_cancel[1],bid_cross_limit[1]))

    if vmap_tests:
        vasks=jnp.array([asks]*nvmap)
        vbids=jnp.array([bids]*nvmap)
        vtrades=jnp.array([trades]*nvmap)
        vlimitms=jnp.array([limitmsg]*nvmap)
        vcancelms=jnp.array([cancelmsg]*nvmap)
        vmatchms=jnp.array([matchmsg]*nvmap)

        out,_=jax.vmap(job.cond_type_side,((0,0,0),0))((vasks,vbids,vtrades),vlimitms)
        res=np.array(timeit.repeat('val=jax.vmap(job.cond_type_side,((0,0,0),0))((vasks,vbids,vtrades),vlimitms); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
        mu=np.mean(res/n_runs)
        sigma=np.std(res/n_runs)
        print("VMAP limit order for book of size ",i,":",mu,"Stdev: ", sigma)
        out,_=jax.vmap(job.cond_type_side,((0,0,0),0))(out,vcancelms)
        res=np.array(timeit.repeat('val=jax.vmap(job.cond_type_side,((0,0,0),0))(out,vcancelms); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
        mu=np.mean(res/n_runs)
        sigma=np.std(res/n_runs)
        print("VMAP cancel order for book of size ",i,":",mu,"Stdev: ", sigma)
        out,_=jax.vmap(job.cond_type_side,((0,0,0),0))(out,vmatchms)
        res=np.array(timeit.repeat('val=jax.vmap(job.cond_type_side,((0,0,0),0))(out,vmatchms); jax.block_until_ready(val)',number=n_runs,repeat=n_repeats,globals=globals()))
        mu=np.mean(res/n_runs)
        sigma=np.std(res/n_runs)
        print("VMAP matched limit order for book of size ",i,":",mu,"Stdev: ", sigma)

print("\n Bid side initial state.")
print(outs[0][0])
print("\n Bid side after adding an order.")
print(outs[0][1])
print("\n Bid side after cancellation")
print(outs[0][2])
print("\n Bid side after incoming sell limit order ")
print(outs[0][3])
    


Limit order for book of size  10 : 0.00017450432553887366 Stdev:  1.013325834942919e-06
Cancel order for book of size  10 : 0.0001717746581882238 Stdev:  4.448049993104346e-06
Matched limit order for book of size  10 : 0.00037140230461955067 Stdev:  6.6084573509849155e-06
Limit order for book of size  100 : 0.00020554061084985734 Stdev:  3.344121854771959e-06
Cancel order for book of size  100 : 0.0001678558513522148 Stdev:  8.687656613863355e-07
Matched limit order for book of size  100 : 0.00037078650370240206 Stdev:  5.561147694871992e-06
Limit order for book of size  1000 : 0.00023297717645764352 Stdev:  3.071366146958367e-06
Cancel order for book of size  1000 : 0.00016859753169119358 Stdev:  9.227914462884426e-07
Matched limit order for book of size  1000 : 0.00037017613537609575 Stdev:  4.310730917647502e-06

 Bid side initial state.
[[2204242     494    1001    1001   34201     663]
 [2209558     456    1003    1003   34203   13163]
 [2204104     465    1005    1005   34203   2